In [1]:
# %cd ./filelists/CUB/
# !ln -s  ../../../CUB_200_2011/images ./images
# %cd ../../

!ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images

# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
!ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

In [9]:
!python wandb_restore.py --id 23ybghei --path ckpts/dogs/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/best_model.tar

wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210607_183814-23ybghei
wandb: Resuming run dogs protonet
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/23ybghei
wandb: Run `wandb off` to turn off syncing.

Checkpoint restored in the given path

wandb: Waiting for W&B process to finish, PID 1947
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20210607_183814-23ybghei/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20210607_183814-23ybghei/logs/debug-internal.log
wandb: Run summary:
wandb:      val/acc 82.0875
wandb:     _runtime 31

In [12]:
# Do again with 200 dimensions
# Do again with 399.tar

!python save_features.py --dataset dogs --train_aug --method protonet --committed

Commit after saving features and testing.
checkpoint_dir: ckpts/dogs/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010
USE BN: True
outfile is features/dogs/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
0/80
10/80
20/80
30/80
40/80
50/80
60/80
70/80


In [13]:
!python test.py --dataset dogs --train_aug --method protonet --committed

Testing involves fine-tuning. Commit after testing.
novel_file features/dogs/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
600 Test Acc = 83.52% +- 0.54%
